In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!readlink -f .

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/cpa-reproducibility/notebooks/figure4


In [3]:
import multiprocessing
from multiprocessing import Process
from multiprocessing import Manager
import threading
from threading import Thread
import utils

### This mimics the approach by Norman et al to obtain coefficients, given data

### Load new data

In [4]:
means = utils.get_means()

True /storage/groups/ml01/workspace/mo/for_nacho/Norman2019_prep_new.h5ad
True /storage/groups/ml01/workspace/carlo.dedonno/cpa-reproducibility/fig4_predicted_adata.h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


True /storage/groups/ml01/workspace/carlo.dedonno/cpa-reproducibility/figure4_latent_adata.h5ad

dimensions of embedding: (497800, 64)
all good. Removing datasets based on repetition is then tractable


/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/cpa-reproducibility/notebooks/figure4/utils.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  means.obs['is.%s' % gene_name] = ((means.obs['a'] == gene_name) | (means.obs['b'] == gene_name)).astype(int)


In [42]:
print(means.shape)

(5262, 5000)


In [6]:
import multiprocessing
from os.path import exists
import warnings
from itertools import combinations
warnings.filterwarnings("ignore")
means.obs['condition'] = means.obs.index

In [24]:
means.var

""
index
RP11-34P13.8
RP11-54O7.3
SAMD11
PLEKHN1
HES4
...
MT-ND4L
MT-ND4
MT-ND6


In [35]:

from sklearn.linear_model import LinearRegression, TheilSenRegressor
import statsmodels.api as sm
from dcor import distance_correlation, partial_distance_correlation
return_dict = {}
def get_parms(means, a, b, ci):
    if ci % 50 == 0:
        print(ci)
    try:
        va = means[means.obs['condition'].isin({'%s+ctrl' % a, 'ctrl+%s' % a}),:].to_df().mean()
        vb = means[means.obs['condition'].isin({'%s+ctrl' % b, 'ctrl+%s' % b}),:].to_df().mean()
        vab = means[means.obs['condition'].isin({'%s+%s' % (a, b), '%s+%s' % (b, a)}),:].to_df().mean()

        
        ref = means[~means.obs.index.str.contains('\+')].to_df().mean()

        da = va - ref
        db = vb - ref
        dab = vab - ref
        
        # print(a, b)
        # print(va.shape, vb.shape)

        x = pd.DataFrame()
        x['da'] = da
        x['db'] = db
        x = np.array(x)
        y = np.array(dab)

        regr = LinearRegression(fit_intercept=False)

        # print(x.shape, y.shape)
        regr.fit(x, y)
        y_pred = regr.predict(x)
        # rlm = sm.RLM(y.ravel(), x, M=sm.robust.norms.TukeyBiweight()).fit()
        # print('rlm fitting...')

        c1 = regr.coef_[0]
        c2 = regr.coef_[1]
        lin_model_fit = distance_correlation(y_pred, y)
        dominance = np.abs(np.log10(c1 / c2))
        magnitude = (c1 ** 2 + c2 ** 2) ** .5
        single_double_sim = distance_correlation(x, y)
        single_sim = distance_correlation(da, db)

        eq_contrib = min(distance_correlation(da, dab), distance_correlation(db, dab)) / max(distance_correlation(da, dab), distance_correlation(db, dab))
        res = pd.DataFrame([[a, b, c1, c2, lin_model_fit, dominance, magnitude,
                             single_double_sim, single_sim, eq_contrib]])
        res.columns = ['a', 'b', 'c1', 'c2', 'lin_model_fit', 'dominance',
                       'magnitude', 'single_double_sim', 'single_sim', 'eq_contrib']
        return_dict[ci] = res
    except ValueError as err:
        print('Something failed while running query id %i' % ci)
        print(err)

In [ ]:
from ThreadingUtils import ThreadingUtils
import pandas as pd
import numpy as np
bkp_path = '../../data/fitting_coefs_rf_path.tsv.gz'

queries = list(combinations(set(means.obs[['a', 'b']].values.flatten()), r=2))
# len(queries)
queries = [[a, b] for a, b in queries if str(a) != 'nan' and str(b) != 'nan']
# queries = [[a, b] for a, b in queries if a == 'KLF1' or b == 'KLF1']
len(queries)

if True or not exists(bkp_path):
    n = 0
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    queries2 = []
    for a, b in queries:
        if str(a) == 'ctrl' or str(b) == 'ctrl':
            continue
        queries2.append([a, b, n])
        n += 1
        
    queries = queries2
    # queries = queries[:1]
    print('# total queries', len(queries))    
    ThreadingUtils.run(get_parms, [[means] + list(t) for t in queries], n_cores=5)
    res = pd.concat([return_dict[k] for k in return_dict.keys()])
    res.to_csv(bkp_path, compression='gzip')
    
res = pd.read_csv(bkp_path, compression='gzip')